**Installing all the requirements**

In [4]:
pip install -r requirements.txt

  Cloning https://github.com/huggingface/parler-tts.git to /tmp/pip-req-build-vzrbn9e6
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/parler-tts.git /tmp/pip-req-build-vzrbn9e6
  Resolved https://github.com/huggingface/parler-tts.git to commit 8e465f1b5fcd223478e07175cb40494d19ffbe17
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 15.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using 

**Loading & finding the audio channel and sampling rate**

In [8]:
from pydub import AudioSegment
audio = AudioSegment.from_file("Recording (5).wav")        # loading of audio file
print(f"sample rate of original = {audio.frame_rate}")     # finding frame rate/sample rate
if audio.channels == 1:                                    # finding audio channel :: 1 for mono & 2 for stereo type
    print("Audio channel = Mono")
else:
    print("Audio channel = Stereo")

FileNotFoundError: [Errno 2] No such file or directory: 'Recording (3).wav'

### **De-sampling & setting audio channel to mono  **

In [18]:
audio = audio.set_frame_rate(16000)                     # de-sampling the frame rate
audio = audio.set_channels(1)                           # setting audio channel to mono
audio.export("pre-processed_audio.wav", format = "wav") # saving file as pre-processed

<_io.BufferedRandom name='pre-processed_audio.wav'>

**checking frame rate & audio channel of pre-processed file**

In [19]:
audio = AudioSegment.from_file("pre-processed_audio.wav")   # loading the newly made audio file
print("Sample rate & audio channel after pre-processing")
print(f"Sample Rate = {audio.frame_rate} KHz")              # finding frame rate/sample rate
if audio.channels == 1:                                     # finding audio channel :: 1 for mono & 2 for stereo type
    print("Audio channel = Mono")
else:
    print("Audio channel = Stereo")

Sample rate & audio channel after pre-processing
Sample Rate = 16000 KHz
Audio channel = Mono


**TRanscribing for later evaluation**

In [21]:
# convetring of the de-sampled audio for checking
text = model.transcribe("pre-processed_audio.wav")
print(text["text"])

/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 can light bend around the corners.


### VAD part: extracting the speeched part form the original part

In [1]:
import whisper
import numpy as np
from pydub import AudioSegment
import torch

model = whisper.load_model("base")     # loading whisper model

# function to perfom the VAD identifying which parts of audio contains speech
def simple_vad(audio, threshold=0.5, min_silence_duration=0.3, sample_rate=16000):

    frame_length = int(sample_rate * 0.01)  # number of smaples in the 10ms frame
    # Calculates the energy for each frame bys umming and squaring so to neglech any -ve values
    energy = np.array([sum(abs(audio[i:i+frame_length]**2)) for i in range(0, len(audio), frame_length)])

    threshold = threshold * np.max(energy)    # dynamic threshold acocording to the lloudness/enegy values of the file

    is_speech = energy > threshold        # array to tell if particular part of speech have voice or not

    # merge short silence gaps
    min_silence_frames = int(min_silence_duration / 0.01)
    for i in range(len(is_speech) - min_silence_frames):
        if sum(is_speech[i:i+min_silence_frames]) > 0:
            is_speech[i:i+min_silence_frames] = True

    return is_speech

def transcribe_with_whisper_vad(audio_file, vad_threshold=0.5, min_silence_duration=0.3):
    print("Loading audio...")
    audio = whisper.load_audio(audio_file)

    print("Performing VAD...")
    is_speech = simple_vad(audio, threshold=vad_threshold, min_silence_duration=min_silence_duration)

    # Apply VAD mask to audio
    speech_audio = audio[np.repeat(is_speech, len(audio) // len(is_speech))]

    print("Starting transcription...")
    try:
        result = model.transcribe(speech_audio, fp16=False)
        print("Transcription completed.")
        return result["segments"]
    except Exception as e:
        print(f"Error during transcription: {e}")
        return []

# Use the function
audio_file = "pre-processed_audio.wav"
transcribed_segments = transcribe_with_whisper_vad(audio_file, vad_threshold=0.01, min_silence_duration=0.3)

if transcribed_segments:
    for segment in transcribed_segments:
        print(f"[{segment['start']:.2f}s - {segment['end']:.2f}s]: {segment['text']}")
        input_text = segment["text"]
else:
    print("No transcribed segments were returned.")

Loading audio...
Performing VAD...
Starting transcription...
Transcription completed.
[0.00s - 2.00s]:  can light bend around the corners.


In [23]:
print(input_text)

 can light bend around the corners.


### USING LLM

In [37]:
from transformers import pipeline

# Using gpt-2 medium LLM model
generator = pipeline('text-generation', model='gpt2-medium')

# function to clean the output provided by model
def clean_response(result, prompt, question):
    clean_response = result.replace(prompt,"").replace(question,"").strip()    # removing content of prompt and question
    return clean_response

# fucntionto get the response form llm
def generate_response(input_text):
    prompt = f"give the simple direct answer without experimentaion for the question: {input_text}"
    response = generator(prompt, max_length=80, num_return_sequences=1, truncation = True)
    return prompt, response[0]['generated_text']

input = input_text
prompt, output = generate_response(input)
cleaned = clean_response(output, prompt, input)   # calling the clean_response function
print(f"Input: {input}")                          # printing input
print(f"Response: {cleaned}")                     # printin the cleaned output : by removing the prompt and question

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Input:  can light bend around the corners.
Response: This is one of the big problems, that even if we think about straight lines and circles and things of the sort,  almost everything has to bend in some way for it to conform to the light.  That's why you can make things like balls spin around on top of each other


**TEXT TO SPEECH CONVERSION**

In [38]:
import torch
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
import soundfile as sf

device = "cuda:0" if torch.cuda.is_available() else "cpu"

# loads the pre-trained model parler tts
model = ParlerTTSForConditionalGeneration.from_pretrained("parler-tts/parler-tts-large-v1").to(device)

# laoding tokenizer to convert text into tokens
tokenizer = AutoTokenizer.from_pretrained("parler-tts/parler-tts-large-v1")

prompt = cleaned     # text output from LLM
# the adjustment of the voice type, pitch and speed can be adjusted with the help of description
description = "Jon's voice is monotone yet slightly fast in delivery, with a very close recording that almost has no background noise."

# to conovert the decription into tokens
input_ids = tokenizer(description, return_tensors="pt").input_ids.to(device)

# converts text into Tokens
prompt_input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

# inputs to the tts model
generation = model.generate(input_ids=input_ids, prompt_input_ids=prompt_input_ids)

# preproceesing for the file to be saved
audio_arr = generation.cpu().numpy().squeeze()

# saving the output as ausio file
sf.write("parler_tts_out1.wav", audio_arr, model.config.sampling_rate)


/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
